# Lab 4, Árbol de decisiones

In [1]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

## Carga de datos

In [2]:
DatosDrogas = pd.read_csv('drug200.csv',delimiter = ',')
DatosDrogas.sample(10)

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
103,56,M,NORMAL,HIGH,8.966,drugX
18,23,M,LOW,HIGH,7.298,drugC
173,41,F,LOW,NORMAL,18.739,drugY
93,42,F,LOW,NORMAL,29.271,drugY
47,68,M,LOW,HIGH,10.291,drugC
73,41,F,NORMAL,NORMAL,22.905,drugY
14,50,F,NORMAL,HIGH,12.703,drugX
12,43,M,LOW,HIGH,15.376,drugY
192,72,M,LOW,HIGH,16.310,drugY
176,48,M,HIGH,NORMAL,10.446,drugA


Cuál es el tamaño del set de datos?

In [3]:
print(f'El dataset consta de {DatosDrogas.size} entradas, las cuales están clasificadas por {DatosDrogas.shape[1]} columnas, y {DatosDrogas.shape[0]} filas.')


El dataset consta de 1200 entradas, las cuales están clasificadas por 6 columnas, y 200 filas.


## Pre-procesamiento de datos

In [4]:
DrogaValores = DatosDrogas[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
DrogaValores[0:5]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.114],
       [28, 'F', 'NORMAL', 'HIGH', 7.798],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

In [5]:
from sklearn import preprocessing

### se cambia los caracteres para femenino y masculino por números
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
DrogaValores[:,1] = le_sex.transform(DrogaValores[:,1])

DrogaValores[0:5]

array([[23, 0, 'HIGH', 'HIGH', 25.355],
       [47, 1, 'LOW', 'HIGH', 13.093],
       [47, 1, 'LOW', 'HIGH', 10.114],
       [28, 0, 'NORMAL', 'HIGH', 7.798],
       [61, 0, 'LOW', 'HIGH', 18.043]], dtype=object)

In [6]:
### Se reclasifica la presión sanguínea al igual que sex a valores numéricos
le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
DrogaValores[:,2] = le_BP.transform(DrogaValores[:,2])

DrogaValores[0:5]

array([[23, 0, 0, 'HIGH', 25.355],
       [47, 1, 1, 'HIGH', 13.093],
       [47, 1, 1, 'HIGH', 10.114],
       [28, 0, 2, 'HIGH', 7.798],
       [61, 0, 1, 'HIGH', 18.043]], dtype=object)

In [7]:
### Se hace lo mismo con el colesterol

le_COL = preprocessing.LabelEncoder()
le_COL.fit(['NORMAL', 'HIGH'])
DrogaValores[:,3] = le_COL.transform(DrogaValores[:,3]) 

DrogaValores[0:5]


array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.114],
       [28, 0, 2, 0, 7.798],
       [61, 0, 1, 0, 18.043]], dtype=object)

Para obtener la y

In [8]:
y = DatosDrogas['Drug']
y[0:5]

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

In [9]:
DatosDrogas['DrugCat'] = pd.factorize(DatosDrogas['Drug'])[0]
DatosDrogas.head()
y2 = DatosDrogas['DrugCat']

## Configuración árbol de decisión

In [10]:
from sklearn.model_selection import train_test_split

Se obtienen los sets de entrenamiento y de prueba utilizando un split. Se toma el 30% para pruebas

In [11]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(DrogaValores, y, test_size=0.3, random_state=3)
X_trainset

array([[26, 0, 0, 1, 19.161],
       [41, 0, 2, 1, 22.905],
       [28, 0, 2, 0, 19.675],
       [19, 0, 0, 0, 13.313],
       [50, 1, 2, 1, 15.79],
       [24, 1, 2, 0, 25.786],
       [72, 1, 1, 0, 16.31],
       [74, 0, 1, 0, 20.942],
       [37, 0, 1, 1, 12.006],
       [31, 1, 0, 1, 17.069],
       [22, 0, 2, 0, 8.607],
       [20, 0, 2, 1, 9.281],
       [28, 0, 1, 0, 13.127],
       [59, 0, 2, 0, 13.884],
       [15, 1, 0, 1, 17.206],
       [51, 0, 1, 1, 23.003],
       [45, 1, 1, 1, 10.017],
       [33, 0, 1, 0, 33.486],
       [39, 1, 0, 0, 9.664],
       [29, 0, 0, 0, 29.45],
       [60, 1, 2, 0, 15.171],
       [24, 0, 0, 1, 18.457],
       [49, 0, 2, 1, 9.381],
       [37, 1, 1, 1, 8.968],
       [32, 0, 0, 1, 10.292],
       [21, 0, 0, 1, 28.632],
       [23, 1, 2, 0, 12.26],
       [40, 1, 0, 0, 27.826],
       [38, 1, 1, 0, 18.295],
       [47, 1, 1, 1, 30.568],
       [22, 0, 0, 1, 22.818],
       [47, 1, 0, 0, 10.403],
       [30, 0, 2, 0, 10.443],
       [69, 1, 1, 0

### Modelado

In [12]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # muestra los parámetros por omisión

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [13]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

### Predicción

In [14]:
predTree = drugTree.predict(X_testset)

In [15]:
print (predTree [0:5])
print (y_testset [0:5])

['drugY' 'drugX' 'drugX' 'drugX' 'drugX']
40     drugY
51     drugX
139    drugX
197    drugX
170    drugX
Name: Drug, dtype: object


### Evaluación del modelo

In [16]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("Precisión de los Arboles de Decisión: ", metrics.accuracy_score(y_testset, predTree))

Precisión de los Arboles de Decisión:  0.9833333333333333


### Visualización del árbol

In [17]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

ModuleNotFoundError: No module named 'sklearn.externals.six'

In [18]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = DatosDrogas.columns[0:5]
targetNames = DatosDrogas["Drug"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

NameError: name 'StringIO' is not defined